In [21]:
import boto3
import sagemaker
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
container = sagemaker.amazon.amazon_estimator.get_image_uri('us-east-1', "xgboost", "latest")

s3_validation_data = 's3://mastering-ml-aws/chapter4/test-vector-csv/'
s3_train_data = 's3://mastering-ml-aws/chapter4/training-vector-csv/'
s3_output_location = 's3://mastering-ml-aws/chapter14/output/'

In [22]:
sagemaker_model = sagemaker.estimator.Estimator(container,
                                                role,
                                                train_instance_count=1,
                                                train_instance_type='ml.c4.4xlarge',
                                                train_volume_size=30,
                                                train_max_run=360000,
                                                input_mode='File',
                                                output_path=s3_output_location,
                                                sagemaker_session=sess)


In [24]:
sagemaker_model.set_hyperparameters(objective='binary:logistic',
                                    max_depth=5,
                                    eta=0.2,
                                    gamma=4,
                                    min_child_weight=6,
                                    subsample=0.7,
                                    silent=0,
                                    num_round=50)


In [25]:
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter,IntegerParameter

tree_tuner = HyperparameterTuner(estimator=sagemaker_model, 
                                 objective_metric_name='validation:auc',
                                 max_jobs=10,
                                 max_parallel_jobs=3,
                                 hyperparameter_ranges={'lambda': ContinuousParameter(0, 1000),
                                                       'max_depth': IntegerParameter(3,7),
                                                       'eta':ContinuousParameter(0.1, 0.5)})

train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated',
                                        content_type='text/csv', s3_data_type='S3Prefix')

validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated',
                                             content_type='text/csv', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data}

tree_tuner.fit(inputs=data_channels, logs=True)

INFO:sagemaker:Creating hyperparameter tuning job with name: xgboost-190407-1342


In [30]:
tree_tuner.best_training_job()

'xgboost-190407-1342-001-5c7e2a26'

In [53]:
sess.sagemaker_client.describe_training_job(TrainingJobName=tree_tuner.best_training_job())

{'TrainingJobName': 'xgboost-190407-1342-001-5c7e2a26',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:095585830284:training-job/xgboost-190407-1342-001-5c7e2a26',
 'TuningJobArn': 'arn:aws:sagemaker:us-east-1:095585830284:hyper-parameter-tuning-job/xgboost-190407-1342',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://mastering-ml-aws/chapter4/sagemaker/output/xgboost-190407-1342-001-5c7e2a26/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'_tuning_objective_metric': 'validation:auc',
  'eta': '0.4843166334129595',
  'gamma': '4',
  'lambda': '892.5118136129357',
  'max_depth': '7',
  'min_child_weight': '6',
  'num_round': '50',
  'objective': 'binary:logistic',
  'silent': '0',
  'subsample': '0.7'},
 'AlgorithmSpecification': {'TrainingImage': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\]#011train-ma

In [54]:
sess.sagemaker_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName='xgboost-190407-1342')

{'HyperParameterTuningJobName': 'xgboost-190407-1342',
 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:095585830284:hyper-parameter-tuning-job/xgboost-190407-1342',
 'HyperParameterTuningJobConfig': {'Strategy': 'Bayesian',
  'HyperParameterTuningJobObjective': {'Type': 'Maximize',
   'MetricName': 'validation:auc'},
  'ResourceLimits': {'MaxNumberOfTrainingJobs': 10,
   'MaxParallelTrainingJobs': 3},
  'ParameterRanges': {'IntegerParameterRanges': [{'Name': 'max_depth',
     'MinValue': '3',
     'MaxValue': '7',
     'ScalingType': 'Auto'}],
   'ContinuousParameterRanges': [{'Name': 'lambda',
     'MinValue': '0',
     'MaxValue': '1000',
     'ScalingType': 'Auto'},
    {'Name': 'eta',
     'MinValue': '0.1',
     'MaxValue': '0.5',
     'ScalingType': 'Auto'}],
   'CategoricalParameterRanges': []},
  'TrainingJobEarlyStoppingType': 'Off'},
 'TrainingJobDefinition': {'StaticHyperParameters': {'_tuning_objective_metric': 'validation:auc',
   'gamma': '4',
   'min_child_we